In [20]:
import os

import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.utils.dsl_utils import external_input
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen

In [21]:
context = InteractiveContext()

In [27]:
base_dir = os.getcwd()
data_dir = "../data/"
examples = external_input(os.path.join(base_dir, data_dir))
example_gen = CsvExampleGen(input=examples)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 4
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /var/folders/_9/_7pgq8cd62s_m3rc63jjg5v00000gn/T/tfx-interactive-2020-01-26T18_27_22.854027-55zwxkqh/CsvExampleGen/examples/4/train/, split: train, id: 5)
            Artifact(type_name: ExamplesPath, uri: /var/folders/_9/_7pgq8cd62s_m3rc63jjg5v00000gn/T/tfx-interactive-2020-01-26T18_27_22.854027-55zwxkqh/CsvExampleGen/examples/4/eval/, split: eval, id: 6)]
        ))

In [29]:
""" Loading from GCP bucket
"""

# examples = external_input("gs://example_compliance_data/")
# example_gen = CsvExampleGen(input=examples)

# context.run(example_gen)

' Loading from GCP bucket\n'

In [30]:
""" Ingesting data from GCP BigQuery
"""

# !export GOOGLE_APPLICATION_CREDENTIALS=/Users/user/credentials.json
# or
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/user/credentials.json"

# from tfx.components.example_gen.big_query_example_gen.component import BigQueryExampleGen  

# query = """SELECT * FROM `complaint-pipeline.consumer_complaints_22Jan.complaints`"""
# example_gen = BigQueryExampleGen(query=query)

# context.run(example_gen)

' Ingesting data from GCP BigQuery\n'

In [31]:
""" Three-way data split
"""

# from tfx.utils.dsl_utils import external_input
# from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
# from  tfx.proto import example_gen_pb2

# base_dir = os.getcwd()
# data_dir = "../data/"
# output = example_gen_pb2.Output( 
#              split_config=example_gen_pb2.SplitConfig(splits=[
#                  example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
#                  example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
#                  example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)
#              ]))

# examples = external_input(os.path.join(base_dir, data_dir)) 
# example_gen = CsvExampleGen(input=examples, output_config=output)

# context.run(example_gen)

' Three-way data split\n'

In [32]:
for artifact in example_gen.outputs['examples'].get():
    print(artifact)

Artifact(type_name: ExamplesPath, uri: /var/folders/_9/_7pgq8cd62s_m3rc63jjg5v00000gn/T/tfx-interactive-2020-01-26T18_27_22.854027-55zwxkqh/CsvExampleGen/examples/4/train/, split: train, id: 5)
Artifact(type_name: ExamplesPath, uri: /var/folders/_9/_7pgq8cd62s_m3rc63jjg5v00000gn/T/tfx-interactive-2020-01-26T18_27_22.854027-55zwxkqh/CsvExampleGen/examples/4/eval/, split: eval, id: 6)


In [33]:
""" Input example
"""

# import os
# from tfx.utils.dsl_utils import external_input
# from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
# from  tfx.proto import example_gen_pb2

# from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
# context = InteractiveContext()

# base_dir = os.getcwd()
# data_dir = "../data/"

# input = example_gen_pb2.Input(splits=[
#                 example_gen_pb2.Input.Split(name='train', pattern='train/*'),
#                 example_gen_pb2.Input.Split(name='eval', pattern='eval/*'),
#                 example_gen_pb2.Input.Split(name='test', pattern='test/*')
#             ])

# examples = external_input(os.path.join(base_dir, data_dir)) 
# example_gen = CsvExampleGen(input=examples, input_config=input)
# context.run(example_gen)

' Input example\n'

In [34]:
""" Span Loading Example
"""

# from tfx.utils.dsl_utils import external_input
# from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
# from  tfx.proto import example_gen_pb2

# base_dir = os.getcwd()
# data_dir = "../data3/"

# input = example_gen_pb2.Input(splits=[
#                 example_gen_pb2.Input.Split(pattern='export-{SPAN}/*')
#             ])

# examples = external_input(os.path.join(base_dir, data_dir)) 

# example_gen = CsvExampleGen(input=examples, input_config=input)
# context.run(example_gen)

' Span Loading Example\n'

In [35]:
from tfx.components.statistics_gen.component import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:214: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [37]:
from tfx.components.schema_gen.component import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'col_0',INT,required,,-
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'complaint_id',INT,required,,-
'consumer_disputed',STRING,optional,single,'consumer_disputed'
'date_received',STRING,required,,'date_received'
'date_sent_to_company',STRING,required,,'date_sent_to_company'
'issue',STRING,optional,single,'issue'
'product',STRING,required,,'product'


,Values
Domain,
'company_response',"'Closed', 'Closed with explanation', 'Closed with monetary relief', 'Closed with non-monetary relief', 'In progress', 'Untimely response'"
'consumer_disputed',"'No', 'Yes'"
'date_received',"'2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-01', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-15', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19'"
'date_sent_to_company',"'2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-01', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-15', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect exchange rate', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender damaged or destro

In [38]:
from tfx.components.example_validator.component import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'sub_product',Unexpected string values,Examples contain values missing from the schema: Traveler\'s/Cashier\'s checks (<1%).
'issue',Unexpected string values,"Examples contain values missing from the schema: Excessive fees (<1%), Lost or stolen check (<1%), Overdraft, savings or rewards features (<1%)."
